In [13]:
# Input
import numpy as np
def map2int(ls):
    return [int(st) for st in ls]

def read_int_list(f):
    return map2int(f.readline().split(' '))

MAX_LATENCY = 10000
with open('example.in') as f:
    nV, nENP, nREQ, nCAH, max_cap = read_int_list(f)
    ReqCah = np.zeros((nREQ, nCAH + 1), dtype=float)
    EnpCah = np.full((nENP, nCAH + 1), MAX_LATENCY, dtype=float)
    ReqVid = np.zeros(nREQ, dtype=int)
    ReqSize = np.zeros(nREQ)
    i_datacenter = nCAH
    total_req = 0
    VidSize = np.asarray(read_int_list(f))
    
    for i_enp in range(nENP):
        lag, n_cah = read_int_list(f)
        EnpCah[i_enp, i_datacenter] = lag
        for _ in range(n_cah):
            i_cah, cah_lag = read_int_list(f)
            EnpCah[i_enp, i_cah] = cah_lag
    
    for i_req in range(nREQ):
        i_vid, i_enp, times = read_int_list(f)
        total_req += times
        ReqVid[i_req] = i_vid
        ReqSize[i_req] = VidSize[i_vid]
        ReqCah[i_req] = EnpCah[i_enp] * times
        

In [28]:
ReqSol = np.asarray([1, 3, 3, 2])

#Verify the result
CahCap = np.full(nCAH, max_cap)
for i_req, cah in enumerate(ReqSol):
    if cah == i_datacenter: continue
    req_sz = ReqSize[i_req]
    CahCap[cah] -= req_sz
    assert CahCap[cah] >=0

In [31]:
# Evaluate Result
lag = ReqCah[range(nREQ), ReqSol]
lag_no_cah = ReqCah[:, i_datacenter]
gain = sum(lag_no_cah - lag)
avg_gain = gain / total_req
avg_gain

462.5

In [48]:
# Convert to submittable output
CahVids = [set() for i in range(nCAH)]
for i_req, cah in enumerate(ReqSol):
    if cah == i_datacenter: continue
    vid = ReqVid[i_req]
    CahVids[cah].add(vid)

In [49]:
CahVids

[set(), {3}, {1}]